In [1]:
! pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [2]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

 19% 5.00M/25.7M [00:00<00:00, 28.7MB/s]
100% 25.7M/25.7M [00:00<00:00, 102MB/s] 


In [3]:
!unzip imdb-dataset-of-50k-movie-reviews.zip 

Archive:  imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


In [4]:
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
dataset = pd.read_csv('IMDB Dataset.csv')

In [11]:
dataset.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [18]:
pos = dataset[dataset['sentiment']=='positive']
neg = dataset[dataset['sentiment']=='negative']

In [21]:
pos.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive


In [39]:
data = pos.iloc[:5000]

In [40]:
data = pd.concat([data, neg[:5000]], ignore_index=True)

In [41]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,"Petter Mattei's ""Love in the Time of Money"" is...",positive
4,"Probably my all-time favorite movie, a story o...",positive


In [42]:
data.shape

(10000, 2)

In [43]:
import random
data = data.sample(frac = 1)

In [44]:
data.reset_index(inplace=True)

In [45]:
data.drop(['index'], inplace=True, axis=1)

In [46]:
data.head()

,review,sentiment
0,"I went into this film with expectations, from ...",negative
1,One of the worst movies I saw in the 90s. I'd ...,negative
2,Samuel Fuller brings his customary playful and...,positive
3,This movie has everything typical horror movie...,positive
4,This game is one of the best horror/shooter ga...,positive


In [47]:
doc = []

for i,j in zip(data['review'],data['sentiment']):
    word = nltk.word_tokenize(i)
    doc.append((word, j))
    
print(doc[0])
    

(['I', 'went', 'into', 'this', 'film', 'with', 'expectations', ',', 'from', 'the', 'hype', ',', 'that', 'it', 'would', 'be', 'insightful', 'and', 'uplifting', '.', 'Certainly', 'something', 'more', 'than', 'a', 'cheap', 'promotional', 'for', 'the', 'band', '``', 'Wilco', '.', '``', '<', 'br', '/', '>', '<', 'br', '/', '>', 'Instead', 'we', 'get', 'a', 'lot', 'of', 'moping', 'and', 'whining', 'about', '``', 'the', 'process', ',', "''", 'a', 'dishonorable', 'and', 'no', 'doubt', 'one-sided', 'portrayal', 'of', 'one', 'band', 'members', 'who', 'was', 'kicked', 'out', 'by', 'the', 'prima', 'donna', 'lead', 'singer/songwriter', ',', 'a', 'gut-wrenching', 'confession', 'by', 'the', 'fallen', 'member', "'s", 'friend', '--', 'for', 'like', '18', 'years', '--', 'saying', 'the', '``', 'friendship', 'had', 'run', 'its', 'course', ',', "''", 'and', 'this', 'whiny', ',', 'uncompelling', 'story', 'about', 'how', 'one', 'record', 'label', '``', 'hurt', 'their', 'feelings', "''", 'by', 'dumping', 'the

In [48]:
words = []
for i in data['review']:
        for a in (nltk.word_tokenize(i)):
            words.append(a.lower())
     

In [49]:
words[:10]

['i',
 'went',
 'into',
 'this',
 'film',
 'with',
 'expectations',
 ',',
 'from',
 'the']

In [50]:
from nltk.corpus import stopwords
stopwords_english = stopwords.words('english')

import string

In [51]:
clean_words = []
for word in words:
    if word not in stopwords_english and word not in string.punctuation:
        clean_words.append(word)

clean_words[:10]

['went',
 'film',
 'expectations',
 'hype',
 'would',
 'insightful',
 'uplifting',
 'certainly',
 'something',
 'cheap']

In [52]:
len(clean_words)

1305249

In [53]:
from nltk import FreqDist
freq = FreqDist(clean_words)

In [54]:
len(freq)

71434

In [55]:
most_common = freq.most_common(4000)
word_features = [item[0] for item in most_common]

In [56]:
def feature(document):
    doc = set(document)
    
    feature = {}
    
    for word in word_features:
        feature['contains(%s)' % word] = (word in doc)
        
    return feature


In [57]:
feature_set = [(feature(doc), category) for (doc, category) in doc]

In [59]:
len(feature_set)

10000

In [60]:
train = feature_set[:7001]
test = feature_set[7001:]

In [63]:
from nltk import NaiveBayesClassifier
classifier = NaiveBayesClassifier.train(train)

In [64]:
from nltk import classify 
 
accuracy = classify.accuracy(classifier, test)
print (accuracy) 

0.8492830943647882
